# Importing Libraries

In [33]:
from numpy import *
from pandas import *
import itertools
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
import string
from nltk.corpus import stopwords
import pandas as pd

# Importing Data

In [2]:
data = read_csv('news.csv')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [20]:
#take label and text for traing
X = data['text']
y = data['label']

In [21]:
#load spacy for english language
nlp = spacy.load("en_core_web_sm")

In [26]:
#punctuations and stopwords declaration
punc = string.punctuation
stopwords = set(stopwords.words('english'))

# Text Cleaing Process

In [27]:
def text_cleaning(sent):
    doc = nlp(sent)
    
    tokens=[]
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        
        tokens.append(temp)
        
        
    cleaned_token = []
    
    for token in tokens:
        if token not in stopwords and token not in punc:
            cleaned_token.append(token)
    return cleaned_token

# Spliting Data

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

# Initialize a TfidVectorizer

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer(tokenizer = text_cleaning)

# Initialize a PassiveAggressiveClassifier and train model

In [30]:
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier(max_iter=100, C=1.5)

model = Pipeline([('tfid', tfid),('pac', pac)])

model.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfid',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                                 tokenizer=<function text_cleaning at 0x00000114A0971168>,
                                 use_idf=True, vocabulary=None)),
                ('pac',
                 PassiveAggressiveClassifier(C=1.5, average=False,
                                             class_wei

# Calculate Accuracy and make confusion matrix

In [31]:
y_pred = model.predict(X_test)

print(accuracy_score(y_pred, y_test))

0.9518547750591949


In [32]:
confusion_matrix(y_pred, y_test)

array([[592,  25],
       [ 36, 614]], dtype=int64)